<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Loading-and-preprocessing-data" data-toc-modified-id="Loading-and-preprocessing-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Loading and preprocessing data</a></span></li><li><span><a href="#Interpolate-the-zero-curve" data-toc-modified-id="Interpolate-the-zero-curve-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Interpolate the zero curve</a></span></li></ul></div>

# Loading and preprocessing data

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('/Users/antoneyoung/Downloads/optionmetrics/Option Data.csv')
data.head()

,Date,SpotPrice,Expiration,Maturity,ForwardPrice,Strike,Dividend,CallBid,CallOffer,CallMid,PutBid,PutOffer,PutMid
0,1996-01-04,617.7,1996-01-20,14,618.473789,600,0.024605,17.125,17.875,17.5000,1.625,1.875,1.7500
1,1996-01-04,617.7,1996-01-20,14,618.473789,605,0.024605,12.875,13.625,13.2500,2.500,2.750,2.6250
2,1996-01-04,617.7,1996-01-20,14,618.473789,610,0.024605,10.000,10.375,10.1875,3.625,4.000,3.8125
3,1996-01-04,617.7,1996-01-20,14,618.473789,615,0.024605,7.500,8.000,7.7500,5.250,5.500,5.3750
4,1996-01-04,617.7,1996-01-20,14,618.473789,620,0.024605,4.500,4.875,4.6875,6.875,7.125,7.0000


In [3]:
zc = pd.read_csv('/Users/antoneyoung/Downloads/optionmetrics/Zero Curve.csv')

In [4]:
zc.head()

,Date,Days,Rate
0,1996-01-02 00:00:00,9,5.763067
1,1996-01-02 00:00:00,15,5.745902
2,1996-01-02 00:00:00,50,5.673317
3,1996-01-02 00:00:00,78,5.608884
4,1996-01-02 00:00:00,169,5.473762


In [5]:
df_r = zc
df_T = data[['Maturity']]

In [6]:
df = pd.merge(df_r, df_T, left_index = True, right_index = True)
df.head()

,Date,Days,Rate,Maturity
0,1996-01-02 00:00:00,9,5.763067,14
1,1996-01-02 00:00:00,15,5.745902,14
2,1996-01-02 00:00:00,50,5.673317,14
3,1996-01-02 00:00:00,78,5.608884,14
4,1996-01-02 00:00:00,169,5.473762,14


In [7]:
df['Date'] = pd.to_datetime(df['Date'])

In [8]:
df['DayChange'] = df['Date'].diff()!=0.0

In [9]:
df.head()

,Date,Days,Rate,Maturity,DayChange
0,1996-01-02,9,5.763067,14,True
1,1996-01-02,15,5.745902,14,True
2,1996-01-02,50,5.673317,14,True
3,1996-01-02,78,5.608884,14,True
4,1996-01-02,169,5.473762,14,True


# Interpolate the zero curve

In [10]:
date_range = df.index.unique()
N = len(date_range)

In [11]:
df_temp = df.copy()
df_temp.set_index (['Date'], inplace=True)
date_rng_check = df_temp.index.unique()

In [12]:
rates_interp_list = []
for i in date_rng_check:
    t_day = i.strftime('%Y-%m-%d')
    t_day_df = df_temp.loc[t_day]
    xp = t_day_df['Days']
    yp = t_day_df['Rate']
    x = t_day_df['Maturity']
    rates_interp = np.interp(x, xp, yp)
    rates_interp_list.append(rates_interp)

In [13]:
list_1d = [b for a in rates_interp_list for b in a]

In [14]:
data['Rates_interp'] = list_1d

In [15]:
data = data.set_index(['Date'])
data.head(10)

,SpotPrice,Expiration,Maturity,ForwardPrice,Strike,Dividend,CallBid,CallOffer,CallMid,PutBid,PutOffer,PutMid,Rates_interp
Date,,,,,,,,,,,,,
1996-01-04,617.7,1996-01-20,14,618.473789,600,0.024605,17.1250,17.875,17.50000,1.625,1.875,1.7500,5.748763
1996-01-04,617.7,1996-01-20,14,618.473789,605,0.024605,12.8750,13.625,13.25000,2.500,2.750,2.6250,5.748763
1996-01-04,617.7,1996-01-20,14,618.473789,610,0.024605,10.0000,10.375,10.18750,3.625,4.000,3.8125,5.748763
1996-01-04,617.7,1996-01-20,14,618.473789,615,0.024605,7.5000,8.000,7.75000,5.250,5.500,5.3750,5.748763
1996-01-04,617.7,1996-01-20,14,618.473789,620,0.024605,4.5000,4.875,4.68750,6.875,7.125,7.0000,5.748763
1996-01-04,617.7,1996-01-20,14,618.473789,625,0.024605,2.2500,2.500,2.37500,9.250,9.500,9.3750,5.748763
1996-01-04,617.7,1996-01-20,14,618.473789,630,0.024605,1.1875,1.375,1.28125,14.250,15.000,14.6250,5.748763
1996-01-04,617.7,1996-01-20,14,618.473789,635,0.024605,0.6250,0.750,0.68750,18.750,19.500,19.1250,5.748763
1996-01-04,617.7,1996-02-17,42,619.989341,600,0.024605,21.1250,22.125,21.62500,3.875,4.125,4.0000,5.689908
